In [1]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
p = catalog.load('parameters')
features = p["features"]
targets=p["targets"]
data = catalog.load("fom_increased_data")
data

                    INFO     Loading data from 'parameters' (MemoryDataSet)...                  ]8;id=863107;file:///home/jack/miniconda3/envs/sampler/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=636826;file:///home/jack/miniconda3/envs/sampler/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

                    INFO     Loading data from 'fom_increased_data' (CSVDataSet)...             ]8;id=927661;file:///home/jack/miniconda3/envs/sampler/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=661002;file:///home/jack/miniconda3/envs/sampler/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

,r_ext_pAl,r_ext_pMeO,pAl_richness,Pg_f,Tg_Tmax,sim_time,Pg_f_hat,Tg_Tmax_hat,std,interest,coverage,quality,datetime,iteration
0,0.998455,0.752775,0.832825,0.439480,0.861510,NaN,NaN,NaN,NaN,NaN,NaN,not_interesting,NaN,NaN
1,0.884572,0.995134,0.489310,0.590641,0.910139,NaN,NaN,NaN,NaN,NaN,NaN,not_interesting,NaN,NaN
2,0.773458,0.891320,0.651214,0.499086,0.883874,NaN,NaN,NaN,NaN,NaN,NaN,not_interesting,NaN,NaN
3,0.513232,0.514477,0.939324,0.321652,0.782492,NaN,NaN,NaN,NaN,NaN,NaN,interest,NaN,NaN
4,0.304354,0.935717,0.096649,0.685096,0.909678,NaN,NaN,NaN,NaN,NaN,NaN,not_interesting,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,0.094199,0.607781,0.288955,0.312976,0.657603,40.000135,0.313436,0.661582,0.822698,0.016691,1.000000,interest,2024-03-04 12:05:52,29.0
299,0.133807,0.937672,0.406250,0.295385,0.649776,40.000123,0.295849,0.655824,0.499479,0.163237,0.998862,interest,2024-03-04 12:05:52,29.0
300,0.156250,0.593750,0.573500,0.248906,0.625136,40.000185,0.254612,0.625804,0.415763,0.251177,0.868967,interest,2024-03-04 12:05:52,29.0
301,0.187500,0.667853,0.000000,0.599192,0.880053,17.906519,0.539970,0.808991,0.200036,0.770800,0.031257,not_interesting,2024-03-04 12:05:52,29.0


In [3]:
points = data[features].values

In [4]:
import plotly.graph_objects as go
import numpy as np

def plot_points_with_cubes(points, side_length):
    fig = go.Figure()

    # Plot points
    fig.add_trace(go.Scatter3d(
        x=points[:, 0],
        y=points[:, 1],
        z=points[:, 2],
        mode='markers',
        marker=dict(color='blue', size=5),
        name='Points'
    ))

    # Plot cubes
    first_cube = True
    for point in points:
        x, y, z = point
        cube_vertices = np.array([
            [x - side_length / 2, y - side_length / 2, z - side_length / 2],
            [x + side_length / 2, y - side_length / 2, z - side_length / 2],
            [x + side_length / 2, y + side_length / 2, z - side_length / 2],
            [x - side_length / 2, y + side_length / 2, z - side_length / 2],
            [x - side_length / 2, y - side_length / 2, z + side_length / 2],
            [x + side_length / 2, y - side_length / 2, z + side_length / 2],
            [x + side_length / 2, y + side_length / 2, z + side_length / 2],
            [x - side_length / 2, y + side_length / 2, z + side_length / 2]
        ])
        cube_x = cube_vertices[:, 0]
        cube_y = cube_vertices[:, 1]
        cube_z = cube_vertices[:, 2]
        # Edges of the cube
        edges = [
            [0, 1], [1, 2], [2, 3], [3, 0],
            [4, 5], [5, 6], [6, 7], [7, 4],
            [0, 4], [1, 5], [2, 6], [3, 7]
        ]
        for edge in edges:
            fig.add_trace(go.Scatter3d(
                x=[cube_x[edge[0]], cube_x[edge[1]]],
                y=[cube_y[edge[0]], cube_y[edge[1]]],
                z=[cube_z[edge[0]], cube_z[edge[1]]],
                mode='lines',
                line=dict(color='red', width=2),
                name='Cubes',
                showlegend=True if first_cube else False,
                # Group all cube on the same legend "cube"
                legendgroup="Cubes"
            ))
            first_cube = False

    fig.update_layout(
        scene=dict(
            xaxis=dict(title='X', range=[0, 1]),
            yaxis=dict(title='Y', range=[0, 1]),
            zaxis=dict(title='Z', range=[0, 1]),
            aspectratio=dict(x=1, y=1, z=1),
        ),
        title='Points with their respective cubes',
    )
    fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
    fig.update_layout(height=1200, margin=dict(l=0, r=0, b=0, t=0))
    fig.show()



In [5]:
import plotly.graph_objects as go
import numpy as np

def plot_points_with_spheres(points, radius):
    fig = go.Figure()

    # Plot points
    fig.add_trace(go.Scatter3d(
        x=points[:, 0],
        y=points[:, 1],
        z=points[:, 2],
        mode='markers',
        marker=dict(color='blue', size=5),
        name='Points'
    ))

    # Plot spheres
    for point in points:
        x, y, z = point
        # Generate points on the surface of the sphere
        theta = np.linspace(0, 2*np.pi, 100)
        phi = np.linspace(0, np.pi, 100)
        theta, phi = np.meshgrid(theta, phi)
        x_sphere = x + radius * np.sin(phi) * np.cos(theta)
        y_sphere = y + radius * np.sin(phi) * np.sin(theta)
        z_sphere = z + radius * np.cos(phi)
        # Add the sphere surface to the plot
        fig.add_trace(go.Surface(
            x=x_sphere,
            y=y_sphere,
            z=z_sphere,
            opacity=0.5,
            showscale=False,
            colorscale='Reds',
            name='Spheres',
            showlegend=False
        ))


    fig.update_layout(
        scene=dict(
            xaxis=dict(title='X', range=[0, 1]),
            yaxis=dict(title='Y', range=[0, 1]),
            zaxis=dict(title='Z', range=[0, 1]),
            aspectratio=dict(x=1, y=1, z=1),
        ),
        title='Points with their respective spheres',
    )
    fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
    fig.update_layout(height=1200, margin=dict(l=0, r=0, b=0, t=0))
    fig.show()

In [6]:
radius = 0.025
selected_points = points
extend_data = False
if extend_data:
    num_extra = 20
    selected_points = np.concatenate([selected_points, np.random.rand(num_extra, 3)])

In [7]:
#plot_points_with_spheres(selected_points, radius)

In [8]:
from scipy import cluster
from sklearn.cluster import DBSCAN
from vedo import Sphere, Cube

def is_near_cube_border(point, radius):
    # Check distance to each face of the cube
    distances = [
        point[0],          # Distance to the left face (x = 0)
        1 - point[0],      # Distance to the right face (x = 1)
        point[1],          # Distance to the bottom face (y = 0)
        1 - point[1],      # Distance to the top face (y = 1)
        point[2],          # Distance to the back face (z = 0)
        1 - point[2]       # Distance to the front face (z = 1)
    ]

    # Check if any distance is less than the radius
    for dist in distances:
        if dist < radius:
            # Point is near a cube border
            return True

    return False 

def compute_two_sphere_intersection_volume(center1, center2, radius):
    d = np.linalg.norm(center1 - center2)

    h = 2 * radius - d
    volume = np.pi * h ** 2 * (radius - h / 3)
    return volume

def compute_simple_case_volume(cluster_points, radius, out_of_cube):
    sphere_volume = (4 / 3) * np.pi * radius ** 3
    acum_volume = 0
    if len(cluster_points) == 1:
        # One sphere case
        acum_volume += sphere_volume
    elif len(cluster_points) == 2:
        # There is a simple equation for the volume of intersection of two spheres
        intersection = compute_two_sphere_intersection_volume(cluster_points[0], cluster_points[1], radius)
        volume = 2 * sphere_volume - intersection
        acum_volume += volume

    return acum_volume

# Function to compute volume of intersection within a cluster
def compute_cluster_volume(cluster_points, radius, out_of_cube, cluster_id):
    if len(cluster_points) <= 2 and not out_of_cube:
        return compute_simple_case_volume(cluster_points, radius, out_of_cube)

    # if len(cluster_points) >= 2:
    #     distances = []
    #     for point in cluster_points:
    #         # measure distance between point and cluster_points
    #         distances.append(np.linalg.norm(cluster_points - point, axis=1))

    #     distances = np.array(distances)
    #     distances[distances == 0] = 1

    #     print("Distances:", distances)
    #     # Filter points in cluster_points with distance less than 1e-4
    #     cluster_points = cluster_points[np.min(distances, axis=0) > 1e-4]

    #     if len(cluster_points) <= 2 and not out_of_cube:
    #         return compute_simple_case_volume(cluster_points, radius, out_of_cube)
        

    # Three or more spheres case
    spheres = []
    for center in cluster_points:
        spheres.append(Sphere(pos=center, r=radius))


    
    cluster_union = spheres[0]
    if len(spheres) > 1:
        for other_sphere in spheres[1:]:
            cluster_union = cluster_union.boolean("plus", other_sphere)


    
    if out_of_cube:
        print("Is out of cube!")
        # Remove volume outside the [0, 1]^3 cube
        center = [0.5]*3
        in_range_cube = Cube(pos=center, side=1.0)
        cluster_union = cluster_union.boolean("intersect", in_range_cube)
    
    return cluster_union.volume()

# Function to compute total volume of intersected cubes
def total_volume(points, radius):
    # Remove duplicated points (they occupy the same volume)
    points = points.round(3)
    points = np.unique(points, axis=0)

    # Clustering of touching spheres
    max_distance = 2 * radius - 1e-6
    dbscan = DBSCAN(eps=max_distance,
                    min_samples=1)
    clusters = dbscan.fit_predict(points)
    
    print("Clusters:\n", clusters)
    print("Unique clusters:\n", np.unique(clusters))

    
    acum_volume = 0

    for cluster_id in np.unique(clusters):
        cluster_points = points[clusters == cluster_id]

        out_of_cube = any([is_near_cube_border(point, radius) for point in cluster_points])

        # TODO: Add cases with out of cube for 1 and 2 spheres.
        
        # 3 or more spheres case
        print(f"Cluster {cluster_id} has {len(cluster_points)} points")
        print(cluster_points)

        acum_volume += compute_cluster_volume(cluster_points, radius, out_of_cube, cluster_id)
        print("-"*70)
            

    return acum_volume

In [9]:
# Calculate total volume with intersections
total_vol_with_intersections = total_volume(selected_points, radius)
print("-"*70)
print(f"Total volume of cubes with intersections: {total_vol_with_intersections:.2e}", )
print(f"Total volume of spheres: {len(selected_points) * (4 / 3) * np.pi * radius ** 3:.2e}")

Clusters:
 [  0   1   2   2   2   3   2   4   5   6   7   7   8   9  10  11  11  12
  13  14  15  16  17   4   4   4  18   4   4  18  19  20  21  22  23  24
  25  26  27   4  28   4   8   4   4  21   4  29  30  31  32   4   4  32
   4   4  33   4  34  29   4   4  35  36  37   4   4  38   4   4   4   4
  39  40  41   4   4  42  43  44  45  46  47  48  49  47   4  50  51  51
  40  40   4  40  52  53  40   4   4   4   4  54  55  40  56  57   4  58
  59  60  61   4  62  43  63  40  64   4  65   4   4  66  67  68   4  43
  69   4   4  70  71  72   4  73  74  75  76  77   4  78  79  79  78  80
  81  82  83  84  85  86  87  88  79  89  90  91  92  93  94  95  96  97
  98  99 100 101 102 103 104 105 106 107 108 109 108 110 111 112 113 114
 115 116 117 118 119 118 120 121 122 123 124 125 126 127 128 129 130 131
 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149
 150 151 152 153 154 155 156 157 158 159 160 161 162 163 162 164 165 166
 167 168 169 170 171 170 172 173 174 175

2024-03-05 12:13:41.872 (   2.318s) [    7F1A6EC64440]    vtkPointLocator.cxx:857    ERR| vtkPointLocator (0x5874e90): No points to subdivide
2024-03-05 12:13:41.872 (   2.318s) [    7F1A6EC64440]vtkIntersectionPolyData:2423  WARN| No Intersection between objects 
2024-03-05 12:13:41.873 (   2.319s) [    7F1A6EC64440]vtkDistancePolyDataFilt:73     ERR| vtkDistancePolyDataFilter (0x58f85e0): No points/cells to operate on
2024-03-05 12:13:41.873 (   2.319s) [    7F1A6EC64440]vtkDistancePolyDataFilt:73     ERR| vtkDistancePolyDataFilter (0x58f85e0): No points/cells to operate on


Is out of cube!
----------------------------------------------------------------------
Cluster 3 has 1 points
[[0.    0.438 0.084]]
Is out of cube!
----------------------------------------------------------------------
Cluster 4 has 42 points
[[0.    0.522 0.156]
 [0.02  0.507 0.108]
 [0.025 0.596 0.139]
 [0.027 0.625 0.125]
 [0.028 0.534 0.134]
 [0.028 0.569 0.125]
 [0.032 0.561 0.188]
 [0.034 0.541 0.162]
 [0.036 0.651 0.187]
 [0.037 0.57  0.157]
 [0.04  0.525 0.187]
 [0.043 0.593 0.197]
 [0.047 0.607 0.167]
 [0.05  0.516 0.164]
 [0.051 0.5   0.211]
 [0.054 0.565 0.196]
 [0.058 0.544 0.215]
 [0.058 0.625 0.199]
 [0.06  0.625 0.236]
 [0.062 0.594 0.219]
 [0.067 0.458 0.223]
 [0.068 0.56  0.255]
 [0.069 0.411 0.23 ]
 [0.069 0.517 0.25 ]
 [0.077 0.603 0.27 ]
 [0.078 0.611 0.298]
 [0.082 0.625 0.277]
 [0.085 0.5   0.31 ]
 [0.091 0.535 0.283]
 [0.091 0.562 0.312]
 [0.091 0.575 0.282]
 [0.093 0.595 0.312]
 [0.094 0.608 0.289]
 [0.097 0.625 0.312]
 [0.104 0.525 0.333]
 [0.107 0.59  0.349]
 

In [10]:
cube_volume(side_length) * len(selected_points)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 cube_volume(side_length) * len(selected_points)                                              │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'cube_volume' is not defined